In [291]:
#!/usr/bin/env python

#Safir Mikhail 18.02.2021 

import os
import json
import random
import time
import numpy
import math
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [300]:
class Settings:
    devices = 10 # Number of devices in the simulation
    signal_per_device_gain = 150 
    signal_per_device_zone = 100
    end_x = 800
    end_y = 800
    
settings = Settings()

In [305]:
class Draw:

    def draw_once(self, points):
        plt.figure(figsize=(10, 10))
        fig = plt.gcf()
        ax = fig.gca()
        
        for i in range(0, len(points[0])):
            ax.add_artist(plt.Circle((points[0][i], points[1][i]), settings.signal_per_device_zone, color='r', fill=False))

        plt.plot(points[0], points[1], 'o')
        plt.plot(0, 0, 'o')
        plt.plot(1000, 1000, 'o')
        
        plt.title("Routing")
        plt.grid(False)
        plt.show();

    
    def draw(self, points):
        
        clear_output(wait=True)
        plt.figure(figsize=(10, 10))
        
        
        fig = plt.gcf()
        ax = fig.gca()
        
        for i in range(0, len(points[0])):
            ax.add_artist(plt.Circle((points[0][i], points[1][i]), settings.signal_per_device_zone, color='r', fill=False))

        plt.plot(points[0], points[1], 'o')
        plt.plot(0, 0, 'o')
        plt.plot(1000, 1000, 'o')
        
        plt.title("Routing")
        plt.grid(False)
        plt.show();

In [306]:
class Generator:
    
    
    def walk(self, points):
        start_points_x = []
        start_points_y = []
        for i in range(0, len(points[0])):
            now_x = points[0][i]
            now_y = points[1][i]
            x, y = random.randint(-3, 3), random.randint(-3, 3)
            now_x += x
            now_y += y
            start_points_x.append(now_x)
            start_points_y.append(now_y)
        return [start_points_x, start_points_y]
            
    
    def start(self):
        start_points_x = []
        start_points_y = []
        for i in range(0, settings.devices):
            x, y = random.randint(100, settings.end_x),  random.randint(100, settings.end_y)
            start_points_x.append(x)
            start_points_y.append(y)
        return [start_points_x, start_points_y]
        pass

In [336]:
class Router:
    
    generator = Generator()
    draw = Draw()
    points = []
    
    def build(self):
        plt.ion()
        points = self.generator.start()
#         self.draw.draw_once(points)
        while True:
            points = self.generator.walk(points)
            self.draw.draw(points)
            plt.pause(0.5)
        pass
    
    def start(self):
        self.build()
        pass

In [339]:
router = Router() # call public class
settings.devices = 50
settings.signal_per_device_zone = 70
router.start()

KeyboardInterrupt: 

<Figure size 720x720 with 0 Axes>

In [338]:
import numpy as np
from scipy.spatial.distance import cdist

x = np.array([[2,1]])
y = np.array([[10,5], [13,2], [60,60]])

d = cdist(x,y)

print(d[0][1])

11.045361017187261
